# 1. Exploratory Network Analysis (ENA)

In this notebook, we perform an exploratory analysis of the raw **MathOverflow answer-to-question** dataset (`sx-mathoverflow-a2q`) from the SNAP database.  
This step precedes any filtering and aims to:

- understand the global structure of the unfiltered interaction graph  
- compute fundamental network statistics  
- inspect temporal activity patterns  
- justify later decisions on subset construction  
- compare raw properties to known results from the literature  

This analysis provides the baseline from which all subsequent processing and structural evaluation will follow.


## 1.1 Dataset Overview

We work with the `sx-mathoverflow-a2q` dataset, which contains **answers to questions**:  
each row represents a timestamped interaction `(u, v, t)`, meaning:

- user **u** answered  
- a question originally posted by **v**  
- at time **t** (UNIX timestamp)

This dataset is a curated subset of the full MathOverflow interaction log, isolating the most meaningful "knowledge-transfer" edges.

According to the SNAP documentation, the dataset includes:

- **21,688 nodes** (unique users)  
- **107,581 temporal answer events**  
- **90,489 static directed edges** (after collapsing duplicates)

In this section, we load the raw dataset and compute initial descriptive statistics.


## 1.2 Loading the Dataset

We begin by loading the raw `sx-mathoverflow-a2q` dataset from the `data/` directory.

The file contains three columns separated by spaces: `source`, `target`, and `timestamp`.



In [ ]:
import pandas as pd

df = pd.read_csv("../data/sx-mathoverflow.txt", delim_whitespace=True, header=None, names=["source", "target", "timestamp"])
df.head()

> A pandas DataFrame makes manipulation easier.


## 1.3 Basic Data Checks

Once loaded, we inspect:

- the shape of the dataset   
- count of unique users  
- total number of interactions

This allows us to confirm that the dataset matches the expected SNAP statistics and ensures data integrity before constructing the network.


In [ ]:
df.shape

In [ ]:
n_users = len(pd.unique(df[['source','target']].values.ravel()))
n_users

In [ ]:
n_events = len(df)
n_events

## 1.4 Temporal Structure of the Interactions

Since each interaction includes a UNIX timestamp, we can extract temporal patterns.

In this section, we:
- convert timestamps into human-readable datetime objects  
- extract year and month  
- compute interaction volume per year  
- visualize how MathOverflow activity evolves over time  

This will later help justify the selection of the **2010–2012** window for the network subset.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from datetime import datetime

df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')

df = df.set_index("datetime")
interactions_by_month = df.resample('ME').size()

plt.figure(figsize=(12, 5))
interactions_by_month.plot()
plt.title("Interações por mês")
plt.ylabel("Quantidade de interações")
plt.show()

## 1.5 Creating the graph via networkX

In [ ]:
G = nx.from_pandas_edgelist(df.reset_index(), source="source", target="target", create_using=nx.DiGraph)

print(f"\nGrafo carregado com {G.number_of_nodes()} nós e {G.number_of_edges()} arestas")

largest_cc = max(nx.weakly_connected_components(G), key=len)
print(f"\nTamanho da maior componente fraca: {len(largest_cc)} nós ({len(largest_cc)/G.number_of_nodes():.2%})")

## 2 Creating the Subset

In [ ]:

# ============================================================
# 1. Subset temporal
# ============================================================

start_date = "2010-01-01"
end_date = "2012-12-31"

subset_by_time = df.loc[
    (df.index >= start_date) & (df.index <= end_date)
]

all_nodes = pd.concat([subset_by_time["source"], subset_by_time["target"]])
unique_users = all_nodes.nunique()
print(f"\nNúmero total de usuários após isolar o dataset temporalmente: {unique_users}")
print(f"\nTotal de interações pós isolar o dataset temporalmente (de {start_date} a {end_date}) : {len(subset_by_time)}")


# ============================================================
# 2. Contagem de interações por utilizador (src + dst)
# ============================================================

interaction_counts = (
    subset_by_time['source'].value_counts() +
    subset_by_time['target'].value_counts()
).fillna(0)

# Selecionar utilizadores com mais de 50 interações
active_users = interaction_counts[interaction_counts > 25].index


# ============================================================
# 3. Filtrar dataset para manter apenas interações entre esses utilizadores
# ============================================================

subset_active = subset_by_time[
    subset_by_time['source'].isin(active_users) &
    subset_by_time['target'].isin(active_users)
]

print(f"\nInterações restantes após filtro por utilizadores ativos: {len(subset_active)}")


# ============================================================
# 4. Criar grafo com utilizadores ativos
# ============================================================

G_active = nx.from_pandas_edgelist(
    subset_active.reset_index(), 'source', 'target', create_using=nx.DiGraph
)

print(f"\nGrafo final contém:")
print(f"- {G_active.number_of_nodes()} nós")
print(f"- {G_active.number_of_edges()} arestas (estáticas)")


# ============================================================
# 5. Maior componente conectada
# ============================================================

largest_cc = max(nx.weakly_connected_components(G_active), key=len)
print(f"\nTamanho da maior componente fraca: {len(largest_cc)} nós "
      f"({len(largest_cc)/G_active.number_of_nodes():.2%})")

cc_active_subgraph = G_active.subgraph(largest_cc)


# ============================================================
# 6. Exportar para GraphML para o Gephi
# ============================================================

nx.write_graphml(cc_active_subgraph, "mathoverflow_active_users_25.graphml")
print("\nSubset exportado como 'mathoverflow_active_users_25.graphml'")

## 3 Analyzing the subset

In [ ]:
import networkx as nx
import numpy as np

print("\n===============================================================")
print("1. PROPRIEDADES DE CONECTIVIDADE E SMALL-WORLDNESS")
print("===============================================================")

N = cc_active_subgraph.number_of_nodes()
E = cc_active_subgraph.number_of_edges()

# 1.1 Coeficiente de Agrupamento Global (Clustering Coefficient)
T = nx.transitivity(cc_active_subgraph)
print(f"Clustering Coefficient): {T:.4f}")


p = nx.density(cc_active_subgraph)
print(f"Density: {p:.4f}")

# 1.2 Comprimento Médio do Caminho (Average Path Length - APL)
try:
    APL = nx.average_shortest_path_length(cc_active_subgraph)
    print(f"Comprimento Médio do Caminho (APL): {APL:.2f}")
except nx.NetworkXError:
    print("APL: Não foi possível calcular o Average Path Length.")
    APL = None


The high transitivity indicates the formation of communities since it proves that users like to interact around the same topics/people. The density indicates that the network is sparse. The lack of a average_shortest_path_length happens because although the graph is fully connected (LWC) it´s also directed, which means that A might be connected to B, and able to transmit information but B isn´t able to the same. 

In [ ]:
G_undirected = cc_active_subgraph.to_undirected(as_view=True)

# 1.2 Comprimento Médio do Caminho (Average Path Length - APL)
try:
    APL = nx.average_shortest_path_length(G_undirected)
    print(f"Comprimento Médio do Caminho (APL): {APL:.2f}")
except nx.NetworkXError:
    print("APL: Não foi possível calcular o Average Path Length.")
    APL = None

In [ ]:
# 2. Análise de Grau e Distribuições
# ============================================================
print("\n===============================================================")
print("2. ANÁLISE DE GRAU")
print("===============================================================")

# Calcular o grau de entrada (respostas recebidas) e saída (respostas dadas)
in_degrees = [d for n, d in cc_active_subgraph.in_degree()]
out_degrees = [d for n, d in cc_active_subgraph.out_degree()]

avg_in_degree = np.mean(in_degrees)
avg_out_degree = np.mean(out_degrees)

print(f"Grau Médio de Entrada (k_in): {avg_in_degree:.2f}")
print(f"Grau Médio de Saída (k_out): {avg_out_degree:.2f}")
print(f"Grau Máximo de Entrada (max k_in): {np.max(in_degrees)}")
print(f"Grau Máximo de Saída (max k_out): {np.max(out_degrees)}")

# Plota a Distribuição de Grau
plt.figure(figsize=(12, 5))

# Distribuição de Grau de ENTRADA (k_in)
plt.subplot(1, 2, 1)
plt.hist(in_degrees, bins=50, log=True, color='skyblue', edgecolor='black')
plt.title('In-Degree Distribuition (log scale)')
plt.xlabel('In-Degree (k_in)')
plt.ylabel('Frequency (log)')

# Distribuição de Grau de SAÍDA (k_out)
plt.subplot(1, 2, 2)
plt.hist(out_degrees, bins=50, log=True, color='lightcoral', edgecolor='black')
plt.title('Out-Degree Distribuition (log scale)')
plt.xlabel('Out-Degree (k_out)')
plt.ylabel('Frequency (log)')

plt.tight_layout()
plt.show()

# Interpretação:
# Se o gráfico em log-log (o que é uma representação comum para esta análise) mostrar uma linha reta,
# isso é um forte indício de uma rede Sem Escala (Scale-Free), regida por uma Lei de Potência.

In [ ]:
import community.community_louvain as community_louvain
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

# Assumindo que 'cc_active_subgraph' é o seu grafo dirigido (DiGraph)
# O algoritmo Louvain funciona melhor em grafos não dirigidos.
# Portanto, trabalhamos com a versão não dirigida do seu subgrafo.
G_undirected = cc_active_subgraph.to_undirected(as_view=True)

# 1. Aplicar o Algoritmo Louvain
# O resultado é um dicionário onde a chave é o nó (usuário) e o valor é o ID da comunidade (0, 1, 2, ...)
partition = community_louvain.best_partition(G_undirected)

# 2. Calcular a Modularidade (Métrica de qualidade do agrupamento)
modularity = community_louvain.modularity(partition, G_undirected)

# 3. Análise da Estrutura
num_communities = max(partition.values()) + 1
community_sizes = pd.Series(partition.values()).value_counts().sort_values(ascending=False)

print("\n==============================================================")
print("1. RESULTADOS DA DETEÇÃO DE COMUNIDADES (LOUVAIN)")
print("==============================================================")
print(f"Modularidade Global (Q): {modularity:.4f}")
print(f"Número Total de Comunidades: {num_communities}")
print(f"Tamanho da Maior Comunidade: {community_sizes.iloc[0]} nós")
print(f"Tamanho da Segunda Maior Comunidade: {community_sizes.iloc[1]} nós")